In [1]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import itertools
from sklearn.utils import shuffle


In [2]:
with open('numpy arrays/X.npy', 'rb') as f:
    X = np.load(f, allow_pickle=True)
    print('X loaded')
with open('numpy arrays/y.npy', 'rb') as f:
    y = np.load(f, allow_pickle=True)
    print('y loaded')

X loaded
y loaded


In [ ]:
X, y = shuffle(X, y, random_state=0)

In [4]:
y[100:200]

array([3, 0, 4, 3, 0, 2, 2, 3, 2, 4, 1, 3, 2, 1, 4, 0, 2, 2, 1, 4, 3, 4,
       2, 0, 3, 3, 0, 2, 4, 4, 3, 4, 3, 3, 2, 2, 2, 4, 3, 4, 4, 3, 1, 0,
       2, 0, 0, 1, 1, 3, 3, 4, 2, 0, 1, 4, 2, 3, 4, 2, 2, 3, 0, 2, 0, 3,
       3, 4, 0, 3, 3, 2, 4, 1, 3, 0, 4, 1, 3, 0, 3, 2, 2, 2, 3, 0, 0, 4,
       2, 4, 2, 4, 2, 1, 1, 1, 4, 0, 2, 4])

In [5]:
X = np.stack((X,)*3, axis=-1)
print(X.shape)

(17615, 100, 100, 3)


In [6]:
new_X = tf.image.resize(
    X, [100,100], method=tf.image.ResizeMethod.BILINEAR, preserve_aspect_ratio=False,
    antialias=False, name=None
)
new_X.shape
X = np.array(new_X)

In [7]:
from tensorflow.keras.applications import VGG16,VGG19,NASNetMobile,DenseNet201
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam 

In [8]:
# Using early stopping and reduce lr on plateau to control the performance
cb_1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=1, min_lr=1e-5, verbose=0) 
cb_2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-6, patience=10, verbose=1, mode='auto')

cbs = [cb_1,cb_2]

In [9]:
from sklearn.model_selection import train_test_split
# split the data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25, random_state = 42, stratify=y) 

In [16]:
model = Sequential()

conv_base = VGG19(input_shape=(100,100,3), include_top=False, pooling='max',weights='imagenet')
# conv_base.trainable = False
model.add(conv_base)
model.add(BatchNormalization())
model.add(Dense(2048, activation='relu', kernel_regularizer=l1_l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(5, activation='softmax')) 

train_layers = [layer for layer in conv_base.layers[::-1][:5]]

for layer in conv_base.layers:
  if layer in train_layers:
    layer.trainable = True

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

In [15]:
model.summary() 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 512)               20024384  
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              1050624   
_________________________________________________________________
batch_normalization_3 (Batch (None, 2048)              8192      
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 10245     
Total params: 21,095,493
Trainable params: 21,090,373
Non-trainable params: 5,120
_________________________________________________________________


In [17]:
history = model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=8, callbacks=cbs,verbose=1) 

Epoch 1/8
 31/413 [=>............................] - ETA: 36:23 - loss: 257.7954 - accuracy: 0.3913